In [4]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("data_titanic_proyecto.csv")

## Analisis de variables

In [5]:
data.head(5)

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,passenger_survived
0,1,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,Lower,M,N
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,Upper,F,Y
2,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Lower,F,Y
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,Upper,F,Y
4,5,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,Lower,M,N


Se busca eliminar las variables especificas y dejar unicamente las generales

In [7]:
columns_to_delete = ['PassengerId', 'Name', 'Ticket']
label_column = ['passenger_survived']

data = data.drop(columns=columns_to_delete)

In [23]:
data.head(5)

,Age,SibSp,Parch,Fare,Cabin,Embarked,passenger_class,passenger_sex,passenger_survived
0,22.0,1,0,7.2500,NaN,S,Lower,M,N
1,38.0,1,0,71.2833,C85,C,Upper,F,Y
2,26.0,0,0,7.9250,NaN,S,Lower,F,Y
3,35.0,1,0,53.1000,C123,S,Upper,F,Y
4,35.0,0,0,8.0500,NaN,S,Lower,M,N


In [24]:
data.describe()

,Age,SibSp,Parch,Fare
count,714.000000,891.000000,891.000000,891.000000
mean,29.699118,0.523008,0.381594,32.204208
std,14.526497,1.102743,0.806057,49.693429
min,0.420000,0.000000,0.000000,0.000000
25%,20.125000,0.000000,0.000000,7.910400
50%,28.000000,0.000000,0.000000,14.454200
75%,38.000000,1.000000,0.000000,31.000000
max,80.000000,8.000000,6.000000,512.329200


Revision de nulos para posibles imputaciones

In [22]:
for col in data.columns:
    cant_nulls = data[col].isna().sum()
    cant_regs = len(data[col])
    print(col)
    print(f'Cantidad de nulos: {cant_nulls}')
    print(f'Porcentage de nulos: {round(cant_nulls/cant_regs*100, 2)}% \n')

Age
Cantidad de nulos: 177
Porcentage de nulos: 19.87% 

SibSp
Cantidad de nulos: 0
Porcentage de nulos: 0.0% 

Parch
Cantidad de nulos: 0
Porcentage de nulos: 0.0% 

Fare
Cantidad de nulos: 0
Porcentage de nulos: 0.0% 

Cabin
Cantidad de nulos: 687
Porcentage de nulos: 77.1% 

Embarked
Cantidad de nulos: 2
Porcentage de nulos: 0.22% 

passenger_class
Cantidad de nulos: 0
Porcentage de nulos: 0.0% 

passenger_sex
Cantidad de nulos: 0
Porcentage de nulos: 0.0% 

passenger_survived
Cantidad de nulos: 0
Porcentage de nulos: 0.0% 



Para concluir que variables son utiles usare un umbral donde si tiene mas del 10% de nulos, la variable sera sacada del dataset de entrenamietno debido a que hacer una imputacion sobre mas del 20% de la data puede llegar a modificar mucho la distribucion de esta. Bajo este criterio las variables Cabin queda fuera del dataset.

In [33]:
data = data.drop(columns=['Cabin'])

Imputacion de valores para variable Age y Embarked

In [29]:
data['Age'].fillna(int(data['Age'].mean()), inplace = True)
data['Embarked'].fillna(data['Embarked'].value_counts().index[0], inplace = True)

In [34]:
for col in data.columns:
    cant_nulls = data[col].isna().sum()
    cant_regs = len(data[col])
    print(col)
    print(f'Cantidad de nulos: {cant_nulls}')
    print(f'Porcentage de nulos: {round(cant_nulls/cant_regs*100, 2)}% \n')

Age
Cantidad de nulos: 0
Porcentage de nulos: 0.0% 

SibSp
Cantidad de nulos: 0
Porcentage de nulos: 0.0% 

Parch
Cantidad de nulos: 0
Porcentage de nulos: 0.0% 

Fare
Cantidad de nulos: 0
Porcentage de nulos: 0.0% 

Embarked
Cantidad de nulos: 0
Porcentage de nulos: 0.0% 

passenger_class
Cantidad de nulos: 0
Porcentage de nulos: 0.0% 

passenger_sex
Cantidad de nulos: 0
Porcentage de nulos: 0.0% 

passenger_survived
Cantidad de nulos: 0
Porcentage de nulos: 0.0% 



Traduccion de variables categoricas a numericas, esto porque no todos los algoritmos que utilizaremos son capaces de recibir como entrada las categorias directamente como es el caso de los arboles de desion.

In [41]:
categorical_variables = ['Embarked', 'passenger_class', 'passenger_sex', 'passenger_survived']
dictionaries = {}

for col in categorical_variables:
    values = data[col].unique()
    dictionaries[col] = {'word_to_number': {values[number]: number for number in range(len(values))}}
    data[col] = data[col].apply(lambda x: dictionaries[col]['word_to_number'][x])
dictionaries

{'Embarked': {'word_to_number': {'S': 0, 'C': 1, 'Q': 2}},
 'passenger_class': {'word_to_number': {'Lower': 0, 'Upper': 1, 'Middle': 2}},
 'passenger_sex': {'word_to_number': {'M': 0, 'F': 1}},
 'passenger_survived': {'word_to_number': {'N': 0, 'Y': 1}}}

In [43]:
data.head(5)

,Age,SibSp,Parch,Fare,Embarked,passenger_class,passenger_sex,passenger_survived
0,22.0,1,0,7.2500,0,0,0,0
1,38.0,1,0,71.2833,1,1,1,1
2,26.0,0,0,7.9250,0,0,1,1
3,35.0,1,0,53.1000,0,1,1,1
4,35.0,0,0,8.0500,0,0,0,0


Revision de distribucion de variables